In [12]:
from kafka import KafkaConsumer
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType
import json
import pyspark
from pyspark.sql.functions import from_json, col
from datetime import datetime
from pyspark.sql.functions import *

In [103]:
spark = SparkSession.builder.master("local").appName("Analytics").getOrCreate()
df = spark.read.option("header","true").csv("2022*/*.csv")
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- country: string (nullable = true)
 |-- name: string (nullable = true)
 |-- number: string (nullable = true)
 |-- CVC: string (nullable = true)
 |-- expire: string (nullable = true)
 |-- IsLegit: string (nullable = true)
 |-- reason: string (nullable = true)



In [104]:
df.count()

1200

In [105]:
df.show(10)

+-------------------+--------------------+------------------+----------------+-------------------+---+------+-------+-------------+
|               date|                  ID|           country|            name|             number|CVC|expire|IsLegit|       reason|
+-------------------+--------------------+------------------+----------------+-------------------+---+------+-------+-------------+
|2022/12/08 16:48:14|ba13df64-770f-11e...|        Costa Rica|Clarence Cabrera|     30457151610835|686| 07/23|   True|Card accepted|
|2022/12/08 16:48:15|ba34aadc-770f-11e...|           Morocco| Lauren Phillips|4236274050401780705|201| 09/30|   True|Card accepted|
|2022/12/08 16:48:15|ba45ca56-770f-11e...|             China|     Thomas Rios|   3511657053669456| 10| 09/32|  False|   False Card|
|2022/12/08 16:48:15|ba573278-770f-11e...|           Estonia|   Mary Thompson|    213161486401808|186| 03/24|  False|   False Card|
|2022/12/08 16:48:15|ba67fb26-770f-11e...|           Comoros|  Vincent Roger

In [106]:
df.groupBy("country").count().orderBy(col("count").desc()).limit(1).show()

+-------+-----+
|country|count|
+-------+-----+
|Bolivia|   12|
+-------+-----+



In [107]:
df = df.withColumn('date', to_timestamp(df.date, 'yyyy/MM/dd HH:mm:ss'))
df = df.withColumn("day", to_date(df.date))
df = df.withColumn("time", date_format(df.date, "HH:mm:ss"))

In [108]:
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- ID: string (nullable = true)
 |-- country: string (nullable = true)
 |-- name: string (nullable = true)
 |-- number: string (nullable = true)
 |-- CVC: string (nullable = true)
 |-- expire: string (nullable = true)
 |-- IsLegit: string (nullable = true)
 |-- reason: string (nullable = true)
 |-- day: date (nullable = true)
 |-- time: string (nullable = true)



In [109]:
df.show()

+-------------------+--------------------+------------------+----------------+-------------------+---+------+-------+-------------+----------+--------+
|               date|                  ID|           country|            name|             number|CVC|expire|IsLegit|       reason|       day|    time|
+-------------------+--------------------+------------------+----------------+-------------------+---+------+-------+-------------+----------+--------+
|2022-12-08 16:48:14|ba13df64-770f-11e...|        Costa Rica|Clarence Cabrera|     30457151610835|686| 07/23|   True|Card accepted|2022-12-08|16:48:14|
|2022-12-08 16:48:15|ba34aadc-770f-11e...|           Morocco| Lauren Phillips|4236274050401780705|201| 09/30|   True|Card accepted|2022-12-08|16:48:15|
|2022-12-08 16:48:15|ba45ca56-770f-11e...|             China|     Thomas Rios|   3511657053669456| 10| 09/32|  False|   False Card|2022-12-08|16:48:15|
|2022-12-08 16:48:15|ba573278-770f-11e...|           Estonia|   Mary Thompson|    213161

In [146]:
def from_date(df, date):
    # date format : YYYY-MM-DD
    return df.filter(df.day == date)

# Get all transaction done after time
def from_time(df, time):
    # time format : HH:mm:ss
    return df.filter(df.time >= time)

def from_date_time(df, date, time):
    return df.filter((df.day == date) & (df.time >= time))

from datetime import datetime, date, timedelta

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return (d2 - d1).days

def from_range(df, start, end):
    total_count = df.filter((df.day >= start) & (df.day <= end)).count()
    list_day = []
    true_count = []
    false_count = []
    nb_day = days_between(start, end)
    for i in range(nb_day + 1):
        date = datetime.strptime(start, '%Y-%m-%d')
        result = date + timedelta(days=i)
        list_day.append(result)
        true_count.append(from_date(df, result).filter(col("IsLegit") == True).count())
        false_count.append(from_date(df, result).filter(col("IsLegit") == False).count())
    return total_count, list_day, true_count, false_count

In [116]:
from_date(df, "2022-12-08").count()

1000

In [112]:
from_time(df, "20:00:00").count()

0

In [111]:
from_date_time(df, "2022-12-08", "16:28:00").count()

1000

In [115]:
from_date(df, "2022-12-07").filter(col("IsLegit") == False).count()

84

In [147]:
a,b,c,d = from_range(df, "2022-12-07", "2022-12-08")

In [158]:
import plotly.express as px

# Create a list of x and y values
x = [1, 2, 3, 4, 5]
y = [10, 20, 30, 40, 50]
y2 = [5,5,5,5,5]
# Create the bar chart
fig = px.bar(x=b, y=[c, d], barmode="group", labels={'x':'Date', 'value':'Count'}, title="Transaction by day")
newnames = {"wide_variable_0":"True", "wide_variable_1":"False"}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name]))
fig.show()